In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/thriller.csv')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv("/content/thriller.csv")
print("Dataset Loaded. Columns:", df.columns)

# Check if 'description' and 'genre' columns exist
if 'description' not in df.columns or 'genre' not in df.columns:
    raise ValueError("Dataset must have 'description' and 'genre' columns")

# Handle missing values
df = df.dropna(subset=['description', 'genre'])

# Convert genres to a list (assuming they are comma-separated)
df['genre'] = df['genre'].apply(lambda x: x.split(','))

# Convert text data into numerical format using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(df['description']).toarray()

# Encode genres using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genre'])

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))


Dataset Loaded. Columns: Index(['movie_id', 'movie_name', 'year', 'certificate', 'runtime', 'genre',
       'rating', 'description', 'director', 'director_id', 'star', 'star_id',
       'votes', 'gross(in $)'],
      dtype='object')
Accuracy: 0.09622411693057248
Classification Report:
              precision    recall  f1-score   support

      Action       0.00      0.00      0.00        55
   Adventure       0.67      0.01      0.01       331
   Animation       0.00      0.00      0.00         0
   Biography       0.00      0.00      0.00        14
      Comedy       0.00      0.00      0.00       212
       Crime       0.50      0.01      0.02       912
       Drama       0.63      0.01      0.02      1842
      Family       0.00      0.00      0.00        54
     Fantasy       0.50      0.00      0.01       230
   Film-Noir       0.00      0.00      0.00        79
     History       0.00      0.00      0.00        73
      Horror       1.00      0.00      0.00       968
       Musi

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [ ]:
#Test with sample descriptions
sample_descriptions = [

    "A group of friends discover an ancient curse in an abandoned mansion.",
    "A hacker fights against a corrupt government agency.",
    "The people of Wakanda fight to protect their home from intervening world powers as they mourn the de...",
    "While vacationing, a girl and her parents are taken hostage by armed strangers who demand that the f...",
    "Dom Toretto and his family are targeted by the vengeful son of drug kingpin Hernan Reyes."
]

sample_features = vectorizer.transform(sample_descriptions).toarray()
sample_predictions = clf.predict(sample_features)

# Ensure at least 3 genres are predicted per sample
threshold = 3 # Define threshold here
sample_pred_adjusted = np.argsort(-sample_predictions, axis=1)[:, :threshold]
adjusted_sample_predictions = np.zeros_like(sample_predictions)
for i, row in enumerate(sample_pred_adjusted):
    adjusted_sample_predictions[i, row] = 1

predicted_genres = mlb.inverse_transform(adjusted_sample_predictions)

# Display predictions
for desc, genres in zip(sample_descriptions, predicted_genres):
    print(f"Description: {desc}\nPredicted Genres: {genres}\n")

Description: A group of friends discover an ancient curse in an abandoned mansion.
Predicted Genres: (' Action', ' Thriller', 'Adventure')

Description: A hacker fights against a corrupt government agency.
Predicted Genres: (' Thriller', 'Action', 'Adventure')

Description: The people of Wakanda fight to protect their home from intervening world powers as they mourn the de...
Predicted Genres: (' Drama', 'Action', 'Animation')

Description: While vacationing, a girl and her parents are taken hostage by armed strangers who demand that the f...
Predicted Genres: (' Action', ' Thriller', 'Adventure')

Description: Dom Toretto and his family are targeted by the vengeful son of drug kingpin Hernan Reyes.
Predicted Genres: (' Action', 'Adventure', 'Animation')

